<a href="https://colab.research.google.com/github/zippy731/wiggle/blob/main/Wiggle_Standalone_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wiggle Standalone 5.1**
---
By [@zippy731](https://twitter.com/zippy731) 

Generates semirandom animation keyframes for zoom/spin/translation for use with [Disco Diffusion v5.1 Colab notebook](https://colab.research.google.com/github/alembics/disco-diffusion/blob/main/Disco_Diffusion.ipynb) created by [@Somnai_dreams](https://twitter.com/Somnai_dreams) and [@gandamu](https://twitter.com/gandamu_ml)

Keyframes generated by Wiggle can also be further edited in the [keyframe string generator](https://keyframe-string-generator.glitch.me/) by [@chigozienri](https://twitter.com/chigozienri)

**Concept:** Wiggle is based on 'episodes' of motion. Each episode is made of three distinct phases: attack (ramp up), decay (ramp down), and sustain (hold level steady). This is similar in concept to an ADSR envelope in a musical synthesizer.

Each parameter will ramp UP to a peak during attack phase, will ramp DOWN to a quiet level during decay phase, and will HOLD the quiet level during sustain phase.  At the end of the sustain phase, another episode will begin, ramping up to the next peak.

The parameters allow you to set the overall duration of each episode, the time split between phases, and the relative levels of the parameters in each phase.

**To Use:** Set parameter ranges and run code. Parameter strings will output to the bottom cell, and can be copied and pasted into Disco Diffusion keyframe inputs.  Wiggle can also be integrated directly into Diffusion notebooks, see code for details.

Setting | Typical | Description 
--- | --- | ---
***Time settings***||
wiggle_frames|1000|Total number of frames to model
preroll_frames|10|Frames before wiggle calcs begin (useful to allow animation to gel)
episode_duration|48|average duration of each episode, in frames
wig_ads_input|(.2,.4,.4)|split between attack,decay,sustain periods,should sum to 1.0
wig_time_var|0.20|allowable variance in time ranges. Must be < 1.0.  Set to 0 for precise control of frames 
***Loop settings***||
wig_loop|False|Force keyframe loop every [n] frames
wig_loop_frames|360|if looping is on, length (in frames) of loop
***Zoom settings***||
wig_zoom_range|(.12,.18)|(2D) min/max peak zoom values. Negative values are zoom out.  In DD41, zoom values range around 1.0, so this zoom range is added to 1.0 later
wig_trz_range|(5,15)|(3D) min/max pixels of z translation (forward zoom) per frame. + is forward, 10 is good starting value.
wig_zoom_quiet_scale_factor|.20|multiplier factor to reduce motion values from peaks to lower 'quiet' period. 0-1, but can be > 1 for 'increase' during quiet period.
***Motion settings***||
wig_angle_range|(-3,3)|(2D) min/max rotation angle range - max degrees per frame
wig_trx_range|(-6,6)|min/max pixels of x translation per frame
wig_try_range|(-2,2)|min/max pixels of y translation per frame
wig_rotx_range|(-3,3)|min/max DEGREES of x rotation(up/down pitch) per frame
wig_roty_range|(-3,3)|min/max DEGREES of y rotation(left/right yaw) per frame
wig_rotz_range|(-3,3)|min/max DEGREES of z rotation(left/right roll) per frame
wig_quiet_scale_factor|.15|multiplier factor for quiet period of all non-zoom parameters
***Glide settings***||
*Glide connects trx and try to rotz and rotx, respectively, like an airplane steers.*||
wig_glide_x_factor|(0,200)|multiplies by rot_z (roll) and adds to existing tr_x 
wig_glide_x_factor|(0,200)|multiplies by rot_x (pitch) and adds to existing tr_y 

**Typical ADS/ settings:**
Different types of motion can be gained with combinations of the ADS timing and the two quiet scale settings

ADS Settings | WZQS | WQS |  Description 
--- | --- | --- | ---
***ADS tweaks***|||
(.2,.4,.4)| 0.20 | 0.15 |default 'purposeful' motion, gliding toward destination, pause at destination
(.1,.5,.4)| 0.20 | 0.15 |'sharp' motion, gliding toward destination, pause at destination
(.3,.3,.4)| 0.20 | 0.15 |meandering motion, gently gliding toward destination, pause at destination
***quiet factor tweaks***|||
(.2,.4,.4)| 0.85 | 0.15 |steady zooming, with periodic turns
(.2,.4,.4)| 3.0 | 0.15 |(use with lower base zoom ranges) - sharp turns with minimal zooming, then faster zooming with less turning

---
Note: The most recent version of Wiggle is always at [Link to Colab](https://colab.research.google.com/github/zippy731/wiggle/blob/main/Wiggle_Standalone_5_1.ipynb)


In [1]:
#======= WIGGLE MODE
#@markdown ---
#@markdown ####**Wiggle:**
#@markdown Generates semirandom keyframes for zoom / spin / translation. 
#@markdown Set ranges below, then run this cell.
#@markdown

#.. can be embedded directly into DD5 notebook.  Copy this code into 
#animation settings tab, just before 'Coherency Settings'
#Then comment out standalone-only code and uncomment 'embedded-only' section.

#standalone-only:
import random
wiggle_frames = 1000#@param {type:"number"}
max_frames = wiggle_frames
#end standalone-only

#embedded-only:
use_wiggle = True #@param {type:"boolean"} 
wiggle_show_params = True #@param {type:"boolean"} 
#end embedded-only code

#@markdown Wiggle preroll and episodes (frames) and duration variability:
preroll_frames = 16#@param {type:"integer"}
episode_duration = 48#@param {type:"integer"}
wig_ads_input = '0.4,0.4,0.4'#@param {type:"string"}
wig_adsmix = [float(x) for x in wig_ads_input.split(',')]
wig_time_var = 0.2#@param {type:"number"}
#@markdown Wiggle loop: force wiggle motion to restart every [n] frames. Useful for motion matching in loops.
wig_loop = False  #@param {type:"boolean"} 
wig_loop_frames = 360#@param {type:"integer"}
#@markdown Wiggle time phases (3 values, sum to 1.0):
#@markdown Zoom (2D) and trz (3D) ranges and quiet factor
wig_zoom_min_max = '0,0'#@param {type:"string"}
wig_zoom_range= [float(x) for x in wig_zoom_min_max.split(',')]
wig_trz_min_max = '6,8'#@param {type:"string"}
wig_trz_range = [int(x) for x in wig_trz_min_max.split(',')]
wig_zoom_quiet_factor =  1.0#@param {type:"number"}# wig_zoom_quiet_scale_factor//scale of zoom quiet periods, as function of above range
#@markdown angle (2D) trx,try(2D/3D) and rotx,roty,rotz (3D) ranges and quiet factor

wig_angle_min_max = '0,0'#@param {type:"string"}
wig_angle_range= [float(x) for x in wig_angle_min_max.split(',')]
wig_trx_min_max = '-2,2'#@param {type:"string"}
wig_trx_range= [float(x) for x in wig_trx_min_max.split(',')]
wig_try_min_max = '-2,2'#@param {type:"string"}
wig_try_range= [float(x) for x in wig_try_min_max.split(',')]

wig_rotx_min_max = '-.5,.5'#@param {type:"string"}
wig_rotx_range= [float(x) for x in wig_rotx_min_max.split(',')]
wig_roty_min_max = '-.5,.5'#@param {type:"string"}
wig_roty_range= [float(x) for x in wig_roty_min_max.split(',')]
wig_rotz_min_max = '-.5,.5'#@param {type:"string"}
wig_rotz_range= [float(x) for x in wig_rotz_min_max.split(',')]
wig_motion_quiet_factor=.5 #@param {type:"number"}
#@markdown GLIDE MODE: tr_x and tr_y yoked to rot_z and rot_x, respectively.
#@markdown *ADDS* to tr_x and tr_y values set above.
# ht @BrokenMindset!
##wig_glide_mode = True #@param {type:"boolean"} 
wig_glide_x_factor = 0 #@param {type:"number"}
wig_glide_y_factor =  0#@param {type:"number"}




if use_wiggle:
    #calculate wiggle keyframes, inject into diffusion notebook  

    #calc time ranges   
    episode_count = round((max_frames)/(episode_duration*.8),0)
    wig_attack_range=(round(episode_duration*wig_adsmix[0]*(1-wig_time_var),0),round(episode_duration*wig_adsmix[0]*(1+wig_time_var),0))
    wig_decay_range=(round(episode_duration*wig_adsmix[1]*(1-wig_time_var),0),round(episode_duration*wig_adsmix[1]*(1+wig_time_var),0))
    wig_sustain_range=(round(episode_duration*wig_adsmix[2]*(1-wig_time_var),0),round(episode_duration*wig_adsmix[2]*(1+wig_time_var),0))
    #------------

    episodes = [(0,1.0,0,0,0,0,0,0,0)] #initialize episodes list
    #ep is: (frame,zoom,angle,trx,try,trz,rotx,roty,rotz)
    episode_starts = [0]
    episode_peaks = [0]
    i = 1
    skip_1 = 0
    wig_frame_count = round(preroll_frames,0)
    while i < episode_count:
      #attack: quick ramp to motion
      if wig_time_var == 0:
        skip_1 = wig_attack_range[0]
      else:
        skip_1 = round(random.randrange(wig_attack_range[0],wig_attack_range[1]),0)
      wig_frame_count += int(skip_1)
      zoom_1 = 1+round(random.uniform(wig_zoom_range[0],wig_zoom_range[1]),3)
      trz_1 = round(random.uniform(wig_trz_range[0],wig_trz_range[1]),3)
      angle_1 = round(random.uniform(wig_angle_range[0],wig_angle_range[1]),3)
      rotx_1 = round(random.uniform(wig_rotx_range[0],wig_rotx_range[1]),3) 
      roty_1 = round(random.uniform(wig_roty_range[0],wig_roty_range[1]),3) 
      rotz_1 = round(random.uniform(wig_rotz_range[0],wig_rotz_range[1]),3) 
      trx_1 = round(random.uniform(wig_trx_range[0],wig_trx_range[1]),3)+round((rotz_1*wig_glide_x_factor),3)
      try_1 = round(random.uniform(wig_try_range[0],wig_try_range[1]),3)+round((rotx_1*wig_glide_y_factor),3)


      episodes.append((wig_frame_count,zoom_1,angle_1,trx_1,try_1,trz_1,rotx_1,roty_1,rotz_1))
      episode_starts.append((wig_frame_count))
      #decay: ramp down to element of interest
      if wig_time_var == 0:
        skip_1 = wig_decay_range[0]
      else:
        skip_1 = round(random.randrange(wig_decay_range[0],wig_decay_range[1]),0)
      wig_frame_count += int(skip_1)
      zoom_1 = 1+(round(wig_zoom_quiet_factor*random.uniform(wig_zoom_range[0],wig_zoom_range[1]),3))
      trz_1 = round(wig_zoom_quiet_factor*random.uniform(wig_trz_range[0],wig_trz_range[1]),3)
      angle_1 = round(wig_motion_quiet_factor*random.uniform(wig_angle_range[0],wig_angle_range[1]),3)
      rotx_1 = round(wig_motion_quiet_factor*random.uniform(wig_rotx_range[0],wig_rotx_range[1]),3)
      roty_1 = round(wig_motion_quiet_factor*random.uniform(wig_roty_range[0],wig_roty_range[1]),3)
      rotz_1 = round(wig_motion_quiet_factor*random.uniform(wig_rotz_range[0],wig_rotz_range[1]),3)
      trx_1 = round(wig_motion_quiet_factor*random.uniform(wig_trx_range[0],wig_trx_range[1]),3)+round((rotz_1*wig_glide_x_factor),3)
      try_1 = round(wig_motion_quiet_factor*random.uniform(wig_try_range[0],wig_try_range[1]),3)+round((rotx_1*wig_glide_y_factor),3)
      episodes.append((wig_frame_count,zoom_1,angle_1,trx_1,try_1,trz_1,rotx_1,roty_1,rotz_1))
      episode_peaks.append((wig_frame_count))
      #sustain: pause during element of interest
      if wig_time_var == 0:
        skip_1 = wig_sustain_range[0]
      else:
        skip_1 = round(random.randrange(wig_sustain_range[0],wig_sustain_range[1]),0)
      wig_frame_count += int(skip_1)
      zoom_1 = 1+(round(wig_zoom_quiet_factor*random.uniform(wig_zoom_range[0],wig_zoom_range[1]),3))
      trz_1 = round(wig_zoom_quiet_factor*random.uniform(wig_trz_range[0],wig_trz_range[1]),3)     
      angle_1 = round(wig_motion_quiet_factor*random.uniform(wig_angle_range[0],wig_angle_range[1]),3)
      rotx_1 = round(wig_motion_quiet_factor*random.uniform(wig_rotx_range[0],wig_rotx_range[1]),3)
      roty_1 = round(wig_motion_quiet_factor*random.uniform(wig_roty_range[0],wig_roty_range[1]),3)
      rotz_1 = round(wig_motion_quiet_factor*random.uniform(wig_rotz_range[0],wig_rotz_range[1]),3)
      trx_1 = round(wig_motion_quiet_factor*random.uniform(wig_trx_range[0],wig_trx_range[1]),3)+round((rotz_1*wig_glide_x_factor),3)
      try_1 = round(wig_motion_quiet_factor*random.uniform(wig_try_range[0],wig_try_range[1]),3)+round((rotx_1*wig_glide_y_factor),3)
      episodes.append((wig_frame_count,zoom_1,angle_1,trx_1,try_1,trz_1,rotx_1,roty_1,rotz_1))
      i+=1

   
    if wig_loop==True and wig_loop_frames > (episode_duration*2):
      #rebuild episode list w repeats
      looping_episodes = [i for i in episodes if i[0] < wig_loop_frames and i[0]>0]
      wig_ep_loop = [(0,1.0,0,0,0,0,0,0,0)] 
      i=0
      while i < (int(max_frames/wig_loop_frames)+1):
        # now update episode list w new starts
        j=0
        while j < len(looping_episodes):
          old_ep = list(looping_episodes[j])
          new_ep_start = old_ep[0]+i*wig_loop_frames
          new_ep = [new_ep_start,old_ep[1],old_ep[2],old_ep[3],old_ep[4],old_ep[5],old_ep[6],old_ep[7],old_ep[8]]
          wig_ep_loop.append(new_ep)
          j+=1
        i+=1
      episodes = wig_ep_loop
      
    #trim off any episode > max_frames
    cleaned_episodes = [i for i in episodes if i[0] < max_frames]
    episodes = cleaned_episodes
    cleaned_episode_starts = [i for i in episode_starts if i < max_frames]
    episode_starts = cleaned_episode_starts
    cleaned_episode_peaks = [i for i in episode_peaks if i < max_frames]
    episode_peaks = cleaned_episode_peaks

    #build full schedule
    keyframe_frames = [item[0] for item in episodes]

    #Build keyframe strings 
    wig_zoom_string=''
    wig_angle_string=''
    wig_trx_string=''
    wig_try_string=''
    wig_trz_string=''
    wig_rotx_string=''
    wig_roty_string=''
    wig_rotz_string=''
    # iterate thru episodes, generate keyframe strings
    ### reformat as keyframe strings for testing
    i = 0
    while i < len(episodes):
      wig_zoom_string += str(int(episodes[i][0]))+':('+str(episodes[i][1])+'),'
      wig_angle_string += str(round(episodes[i][0],0))+':('+str(episodes[i][2])+'),'
      wig_trx_string += str(round(episodes[i][0],0))+':('+str(episodes[i][3])+'),'
      wig_try_string += str(round(episodes[i][0],0))+':('+str(episodes[i][4])+'),'
      wig_trz_string += str(round(episodes[i][0],0))+':('+str(episodes[i][5])+'),'
      wig_rotx_string += str(round(episodes[i][0],0))+':('+str(episodes[i][6])+'),'
      wig_roty_string += str(round(episodes[i][0],0))+':('+str(episodes[i][7])+'),'
      wig_rotz_string += str(round(episodes[i][0],0))+':('+str(episodes[i][8])+'),'
      i+=1    

    zoom = wig_zoom_string
    angle = wig_angle_string 
    translation_x = wig_trx_string
    translation_y =  wig_try_string
    translation_z =  wig_trz_string
    rotation_3d_x = wig_rotx_string
    rotation_3d_y = wig_roty_string
    rotation_3d_z = wig_rotz_string

    if wiggle_show_params:
      print('keyframe transitions:')
      print(keyframe_frames)
      print('episode_starts:')
      print(episode_starts)
      print('episode_peaks:')
      print(episode_peaks)
      print ('angle is')
      print(angle)    
      print ('zoom is')
      print(zoom)
      print ('translation_x is')
      print(translation_x)
      print ('translation_y is')
      print(translation_y)  
      print ('translation_z is')
      print(translation_z)  
      print ('rotation_3d_x is')
      print(rotation_3d_x)  
      print ('rotation_3d_y is')
      print(rotation_3d_y)  
      print ('rotation_3d_z is')
      print(rotation_3d_z)  
      print('end of wiggle params')

#============= END WIGGLE 

keyframe transitions:
[0, 13, 29, 44, 57, 75, 87, 103, 119, 133, 152, 165, 181, 200, 214, 227, 241, 259, 272, 289, 308, 323, 339, 357, 373, 387, 405, 420, 435, 449, 462, 481, 499, 514, 528, 541, 557, 573, 591, 606, 622, 640, 655, 673, 688, 701, 715, 733, 749, 768, 787, 800, 814, 828, 844, 857, 870, 885, 904, 919, 935, 951, 965, 977, 991]
episode_starts:
[0, 13, 57, 103, 152, 200, 241, 289, 339, 387, 435, 481, 528, 573, 622, 673, 715, 768, 814, 857, 904, 951, 991]
episode_peaks:
[0, 29, 75, 119, 165, 214, 259, 308, 357, 405, 449, 499, 541, 591, 640, 688, 733, 787, 828, 870, 919, 965]
angle is
0:(0),13:(0.0),29:(0.0),44:(0.0),57:(0.0),75:(0.0),87:(0.0),103:(0.0),119:(0.0),133:(0.0),152:(0.0),165:(0.0),181:(0.0),200:(0.0),214:(0.0),227:(0.0),241:(0.0),259:(0.0),272:(0.0),289:(0.0),308:(0.0),323:(0.0),339:(0.0),357:(0.0),373:(0.0),387:(0.0),405:(0.0),420:(0.0),435:(0.0),449:(0.0),462:(0.0),481:(0.0),499:(0.0),514:(0.0),528:(0.0),541:(0.0),557:(0.0),573:(0.0),591:(0.0),606:(0.0),622:(0.0),6